# CoppeliaSim: PART 2
* Run the scene workshop1.ttt in CoppeliaSim

In [1]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Robot

In [2]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(self.client_id, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

## Trajectory Generation

In [3]:
def point_to_point_traj(x1, x2, v1, v2, delta_t):
  t = np.linspace(0, delta_t, 500)  
  a0 = x1
  a1 = v1
  a2 = (3*x2 - 3*x1 - 2*v1*delta_t - v2 * delta_t) / (delta_t**2)
  a3 = (2*x1 + (v1 + v2) * delta_t  - 2 * x2) / (delta_t**3)

  polynomial = a0 + a1 * t + a2 * t**2 + a3 * t**3
  derivative = a1 + 2*a2 * t + 3 * a3 * t**2
  return polynomial, derivative, t

In [4]:
def rotate(dur, direction):
    ts = np.linspace(0, dur, 10)  
    for t in ts:
        r.send_motor_velocities([direction,0,0,0])
        time.sleep(dur/10)

# Trajectory follower

A {straight-line trajectory} is a function that describes a segment of a straight line in the Euclidean space that starts at a point $\mathbf{p}_0$ at $t=0$ and ends at a point $\mathbf{p}_f$ at time $t=t_f$. The equation that describes the trajectory is:
	\begin{equation}
	\mathbf{\gamma}(t) = \mathbf{a}_1 t + \mathbf{a}_0,
	\label{eq:line}
	\end{equation}
	where $\mathbf{a}_1=\frac{\mathbf{p}_f - \mathbf{p}_0}{t_f}$ and $\mathbf{a}_0=\mathbf{p}_0$, for $t\in[0,t_f]$. 	

In [5]:
def follow_traj(robot_position, desired_position, vi, vf, tf):
    # Time duration
    # Time interval
    time_steps = linspace(0, tf, 500)

    x1 = robot_position
    x2 = desired_position
    v1 = vi
    v2 = vf
    a0 = x1
    a1 = 0
    a2 = (3*x2 - 3*x1 - 2*v1*tf - v2 * tf) / (tf**2)
    a3 = (2*x1 + (v1 + v2) * tf  - 2 * x2) / (tf**3)

    # a1 = (desired_position - robot_position) / tf
    # a0 = robot_position

    for t in time_steps:
        polynomial = a0 + a1 * t + a2 * t**2 + a3 * t**3
        derivative = a1 + 2*a2 * t + 3 * a3 * t**2
        point_traj = polynomial
        vel_traj = derivative
        #print(point_traj)

        disk.set_position(point_traj)
        #ref_point.set_position(point_traj)

        # Sensing
        robot_position = r.get_position()


        # Trajectory tracker
        u = 10 * (point_traj - robot_position) + vel_traj

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])



        time.sleep(tf/500)
    

In [19]:
def printPos(robot_position, desired_position):
    print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))
    print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

In [24]:
#Joint name in coppelia

motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot
# Connect to the sphere that will be a reference
#ref_point = robot('Sphere11', client_id=r.client_id)
# Initial locations

# print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))


# Virtual disk
disk = robot('Disc', client_id=r.client_id)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere1')
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position, 0, 0, 5)

rotate(5,1)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere0')
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 5)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere')
rotate(5,-1)
printPos(robot_position,desired_position)
follow_traj(robot_position, np.asarray([robot_position[0],robot_position[1]-2,robot_position[2]]).T,0, 0, 5)

# rotate(7.25,1)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere')
desired_position = np.asarray([desired_position[0], desired_position[1]+.25, desired_position[0]])
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 5)

robot_position = r.get_position()
desired_position = np.asarray([7, 1.5, .05])
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 5)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere2')
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 5)

robot_position = r.get_position()
desired_position = np.asarray([9.25, -2.5, .05])
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 5)

robot_position = r.get_position()
desired_position = np.asarray([9.25, -6.15, .05])
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 1)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere3')
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 1)

robot_position = r.get_position()
desired_position = np.asarray([1, -6.15, .05])
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 1)

robot_position = r.get_position()
desired_position = np.asarray([-1, -1, .05])
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 1)

robot_position = r.get_position()
desired_position = np.asarray([-4, -8.5, .05])
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 1)

robot_position = r.get_position()
desired_position = r.get_object_position('Sphere4')
printPos(robot_position,desired_position)
follow_traj(robot_position, desired_position,0, 0, 5)

r.send_motor_velocities([0, 0, 0, 0])
r.close_connection() 

Robot connected
Robot connected
Robot position: (7.62, 8.53) 
Desired position: (0.40, 8.05) 
Robot position: (0.08, 7.88) 
Desired position: (-3.57, 3.40) 
Robot position: (-3.58, 3.39) 
Desired position: (1.93, 2.22) 
Robot position: (-3.58, 1.39) 
Desired position: (1.93, 2.47) 
Robot position: (1.93, 2.48) 
Desired position: (7.00, 1.50) 
Robot position: (7.00, 1.50) 
Desired position: (4.80, -2.50) 
Robot position: (4.80, -2.50) 
Desired position: (9.25, -2.50) 
Robot position: (9.25, -2.50) 
Desired position: (9.25, -6.15) 
Robot position: (9.25, -6.17) 
Desired position: (6.13, -6.15) 
Robot position: (6.11, -6.15) 
Desired position: (1.00, -6.15) 
Robot position: (0.97, -6.16) 
Desired position: (-1.00, -1.00) 
Robot position: (-1.02, -0.98) 
Desired position: (-4.00, -8.50) 
Robot position: (-3.99, -8.54) 
Desired position: (-6.65, -8.50) 
Connection closed
